<a href="https://colab.research.google.com/github/N-T-C/structural/blob/main/AS3600BeamGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#@title Import stuffs
from matplotlib import pyplot as plt
import scipy.stats as stats
import numpy as np
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.optimize as opt

In [16]:
#@title function hehe
def AS3600_2018_RectBeam(D,B,cover,fc,fsy,Asc,Ast):
    E      = 200000
    alpha2 =  max(0.85-0.0015*fc, 0.67) # Rectangular stress block parameter
    gamma  =  min(0.85,max(0.97-0.0025*fc, 0.67)) # Rectangular stress block parameter
    
    conStrain = 0.003
   
    d = D - cover
    
    def calc_ku(ku):
        dn = ku * d
        steelCompStrain = conStrain * (dn-cover) / dn
        steelTenStrain  = conStrain * (d-dn) / dn

        steelCompForce  = min(fsy, E * steelCompStrain) * Asc
        steelTenForce   = min(fsy, E * steelTenStrain) * Ast
        ConcreteForce   = gamma*dn * B * (alpha2 *fc)
        residual = steelCompForce + ConcreteForce - steelTenForce
        return residual
    
    ku = opt.brentq(lambda xi: calc_ku(xi), 0.000000000001, 1)
    
    dn = ku * d
    steelCompStrain = conStrain * (dn-cover) / dn
    steelTenStrain  = conStrain * (d-dn) / dn

    Cs  = min(fsy, E * steelCompStrain) * Asc
    T   = min(fsy, E * steelTenStrain) * Ast
    Cc  = gamma*dn * B * (alpha2 *fc)
    residual = Cs + Cc - T
    
    M = Cs * (dn - cover) + Cc * (dn - gamma*dn / 2) + T * (d-dn)
    
    return M / pow(10,6)

In [17]:
#@title Default title text
def myFun(D, B, cover, fc, fsy, Asc, Ast):
    
    res = AS3600_2018_RectBeam(D, B, cover, fc, fsy, Asc, Ast)
    print('Capacity of section (kNm)')
    print(res)
    def Fun1(x):
        return AS3600_2018_RectBeam(D, B, cover, fc, fsy, Asc, x)
    
    fig = plt.figure()
    ax = plt.axes()
    x = np.linspace(100, 10000, 50)

    Fun1_v = np.vectorize(Fun1)
    y = Fun1_v(x)

    
    ax.set_xlabel('Tensile Reinforcement Ast (mm^2)')
    ax.set_ylabel('Bending Moment Capacity (kNm)')
        
    plt.plot(x, y, color='Indigo', linestyle='--', linewidth=2)
    plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=1)
  
   
    x1 = Ast
    y1 = res
    plt.plot(x1, y1,'g:o', color='red', linestyle='--', linewidth=50)
        
   
   
 
    plt.show()

In [18]:
#@title DMain
interact(myFun,
         D = widgets.IntSlider(min=100, max=1000, step=10, value=500), 
         B = widgets.IntSlider(min=100, max=2000, step=10, value=300), 
         cover = widgets.IntSlider(min=0, max=200, step=1, value=50), 
         fc = widgets.IntSlider(min=10, max=100, step=1, value=32), 
         fsy = {250,300,350,400,450,500}, 
         Asc = widgets.IntSlider(min=0, max=5000, step=1, value=500), 
         Ast = widgets.IntSlider(min=0, max=5000, step=1, value=500))

interactive(children=(IntSlider(value=500, description='D', max=1000, min=100, step=10), IntSlider(value=300, …

<function __main__.myFun>